<a href="https://colab.research.google.com/github/jimbrig-work/Collab-Notebooks/blob/main/Getting_started_with_BigQuery.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Before you begin


1.   Use the [Cloud Resource Manager](https://console.cloud.google.com/cloud-resource-manager) to Create a Cloud Platform project if you do not already have one.
2.   [Enable billing](https://support.google.com/cloud/answer/6293499#enable-billing) for the project.
3.   [Enable BigQuery](https://console.cloud.google.com/flows/enableapi?apiid=bigquery) APIs for the project.


### Provide your credentials to the runtime

In [ ]:
from google.colab import auth
auth.authenticate_user()
print('Authenticated')

## Optional: Enable data table display

Colab includes the ``google.colab.data_table`` package that can be used to display large pandas dataframes as an interactive data table.
It can be enabled with:

In [ ]:
%load_ext google.colab.data_table

If you would prefer to return to the classic Pandas dataframe display, you can disable this by running:
```python
%unload_ext google.colab.data_table
```

# Use BigQuery via magics

The `google.cloud.bigquery` library also includes a magic command which runs a query and either displays the result or saves it to a variable as a `DataFrame`.

In [2]:
# Display query output immediately

%%bigquery --project pg-us-e-app-584489
SELECT 
  COUNT(*) as total_rows
FROM `Equitable.ProRata`

,total_rows
0,1048575


In [4]:
# Save output in a variable `df`

%%bigquery --project pg-us-e-app-584489 df
SELECT * FROM `Equitable.ProRata`

In [5]:
df

,Year_curr,Month_curr,curr_ss_date,Q,Life_Benefit,With_Map,ITMband,product,product_series,legacy_nonleg,Buyout_Indicator,db_benefit,tax_status,channel,pt_nlg_ind,Withdrawal_Taken,ProRata_ind,Rate_ib_db,AAge_band,Venerable_Ind,_49_1_Indicator,Inforce,Inforce_AV,Inforce_BB,Expected_Pro_Rata_Amount,Actual_Pro_Rata_Amount,Expected
0,2016,1,2016-01-29,2016Q1,DB_Only,Other,-50% and Less,ACC,ACC_1995/1996,Legacy,No_Buyout,RAT/ROP,NQ,R,False,0,0,6.0,67,EQH,EQH,1,74559.96,0.0,99.518907,0.0,EI
1,2016,1,2016-01-29,2016Q1,DB_Only,Other,-50% and Less,ACC,ACC_1995/1996,Legacy,No_Buyout,RAT/ROP,NQ,R,False,0,0,6.0,69,EQH,EQH,1,196082.82,0.0,278.176161,0.0,EI
2,2016,1,2016-01-29,2016Q1,DB_Only,Other,-50% and Less,ACC,ACC_1995/1996,Legacy,No_Buyout,RAT/ROP,NQ,R,False,0,0,6.0,75,EQH,EQH,1,41972.32,0.0,70.104267,0.0,EI
3,2016,1,2016-01-29,2016Q1,DB_Only,Other,-50% and Less,ACC,ACC_1995/1996,Legacy,No_Buyout,RAT/ROP,NQ,R,False,0,0,6.0,76,EQH,EQH,1,22135.57,0.0,37.899785,0.0,EI
4,2016,1,2016-01-29,2016Q1,DB_Only,Other,-50% and Less,ACC,ACC_1995/1996,Legacy,No_Buyout,RAT/ROP,NQ,R,False,0,0,6.0,77,EQH,EQH,3,506724.17,0.0,888.836421,0.0,EI
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1048570,2016,2,2016-02-29,2016Q1,DB_Only,Other,40% to 50%,ACC,ACC_Oct. 1996 - pre '02,Legacy,No_Buyout,RAT/ROP,Q,W,False,1,1,0.0,80,EQH,EQH,2,98782.77,0.0,491.551295,1200.0,GAAP
1048571,2016,2,2016-02-29,2016Q1,DB_Only,Other,50% to 60%,ACC,ACC_Oct. 1996 - pre '02,Legacy,No_Buyout,RAT/ROP,NQ,W,False,1,1,0.0,86+,EQH,EQH,1,26420.81,0.0,64.744195,500.0,GAAP
1048572,2016,2,2016-02-29,2016Q1,DB_Only,Other,50% to 60%,ACC,ACC_Oct. 1996 - pre '02,Legacy,No_Buyout,RAT/ROP,NQ,W,False,1,1,0.0,86,EQH,EQH,1,33031.62,0.0,72.592490,202.0,GAAP
1048573,2016,2,2016-02-29,2016Q1,DB_Only,Other,60% to 75%,ACC,ACC_Oct. 1996 - pre '02,Legacy,No_Buyout,RAT/ROP,NQ,W,False,1,1,0.0,79,EQH,EQH,1,32003.47,0.0,62.385431,125.0,GAAP


In [6]:
df.describe()

,Year_curr,Month_curr,Withdrawal_Taken,ProRata_ind,Rate_ib_db,Inforce,Inforce_AV,Inforce_BB,Expected_Pro_Rata_Amount,Actual_Pro_Rata_Amount
count,1048575.0,1.048575e+06,1.048575e+06,1.048575e+06,1.038995e+06,1.048575e+06,1.048575e+06,1.048575e+06,1.048575e+06,1.048575e+06
mean,2016.0,5.911773e+00,4.281811e-01,8.879098e-02,5.599320e+00,4.746727e+00,5.587460e+05,7.982553e+05,3.331600e+02,4.691309e+02
std,0.0,3.654890e+00,4.948154e-01,2.844419e-01,1.444500e+00,1.756150e+01,1.852167e+06,2.457844e+06,1.102753e+03,6.611154e+03
min,2016.0,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00,3.400000e-01,0.000000e+00,8.040000e-05,0.000000e+00
25%,2016.0,2.000000e+00,0.000000e+00,0.000000e+00,6.000000e+00,1.000000e+00,6.646696e+04,8.551785e+04,3.109195e+01,0.000000e+00
50%,2016.0,6.000000e+00,0.000000e+00,0.000000e+00,6.000000e+00,2.000000e+00,1.758268e+05,2.488429e+05,9.329546e+01,0.000000e+00
75%,2016.0,9.000000e+00,1.000000e+00,0.000000e+00,6.000000e+00,4.000000e+00,4.882955e+05,7.116006e+05,2.762332e+02,0.000000e+00
max,2016.0,1.200000e+01,1.000000e+00,1.000000e+00,6.500000e+00,2.305000e+03,2.449158e+08,2.677738e+08,1.490721e+05,1.580695e+06


# Use BigQuery through google-cloud-bigquery

See [BigQuery documentation](https://cloud.google.com/bigquery/docs) and [library reference documentation](https://googlecloudplatform.github.io/google-cloud-python/latest/bigquery/usage.html).

The [GSOD sample table](https://bigquery.cloud.google.com/table/bigquery-public-data:samples.gsod) contains weather information collected by NOAA, such as precipitation amounts and wind speeds from late 1929 to early 2010.


### Declare the Cloud project ID which will be used throughout this notebook

In [7]:
project_id = 'pg-us-e-app-584489'

### Sample approximately 2000 random rows

In [8]:
from google.cloud import bigquery

client = bigquery.Client(project=project_id)

sample_count = 2000
row_count = client.query('''
  SELECT 
    COUNT(*) as total
  FROM `Equitable.ProRata`''').to_dataframe().total[0]

df = client.query('''
  SELECT
    *
  FROM
    `Equitable.ProRata`
  WHERE RAND() < %d/%d
''' % (sample_count, row_count)).to_dataframe()

print('Full dataset has %d rows' % row_count)

Full dataset has 1048575 rows


### Describe the sampled data

In [9]:
df.describe()

,Year_curr,Month_curr,Withdrawal_Taken,ProRata_ind,Rate_ib_db,Inforce,Inforce_AV,Inforce_BB,Expected_Pro_Rata_Amount,Actual_Pro_Rata_Amount
count,1973.0,1973.000000,1973.000000,1973.000000,1960.000000,1973.000000,1.973000e+03,1.973000e+03,1973.000000,1973.000000
mean,2016.0,5.856564,0.443994,0.083122,5.625255,4.692854,5.529788e+05,8.034482e+05,317.284970,379.021135
std,0.0,3.644469,0.496979,0.276137,1.373656,18.515438,1.817637e+06,2.305700e+06,951.009133,3849.573137
min,2016.0,1.000000,0.000000,0.000000,0.000000,1.000000,4.810000e+00,0.000000e+00,0.001261,0.000000
25%,2016.0,2.000000,0.000000,0.000000,6.000000,1.000000,6.768453e+04,8.249324e+04,32.635276,0.000000
50%,2016.0,6.000000,0.000000,0.000000,6.000000,2.000000,1.772030e+05,2.525092e+05,98.213920,0.000000
75%,2016.0,9.000000,1.000000,0.000000,6.000000,4.000000,5.229574e+05,7.709958e+05,283.201533,0.000000
max,2016.0,12.000000,1.000000,1.000000,6.500000,618.000000,5.238140e+07,5.938335e+07,30167.318700,128601.170000


### View the first 10 rows

In [10]:
df.head(10)

,Year_curr,Month_curr,curr_ss_date,Q,Life_Benefit,With_Map,ITMband,product,product_series,legacy_nonleg,Buyout_Indicator,db_benefit,tax_status,channel,pt_nlg_ind,Withdrawal_Taken,ProRata_ind,Rate_ib_db,AAge_band,Venerable_Ind,_49_1_Indicator,Inforce,Inforce_AV,Inforce_BB,Expected_Pro_Rata_Amount,Actual_Pro_Rata_Amount,Expected
0,2016,2,2016-02-29,2016Q1,DB_Only,Other,-50% and Less,ACC,ACC_1995/1996,Legacy,No_Buyout,RAT/ROP,Q,R,False,0,0,6.0,75,EQH,EQH,2,187223.81,0.00,859.201268,0.0,EI
1,2016,7,2016-07-29,2016Q3,DB_Only,Other,0% to 10%,ACC,ACC_1995/1996,Legacy,No_Buyout,RAT/ROP,NQ,R,False,0,0,6.0,83,EQH,EQH,1,71020.72,0.00,142.449809,0.0,EI
2,2016,7,2016-07-29,2016Q3,DB_Only,Other,20% to 30%,ACC,ACC_1995/1996,Legacy,No_Buyout,RAT/ROP,NQ,R,False,0,0,6.0,82,EQH,EQH,2,70919.32,0.00,139.267815,0.0,EI
3,2016,9,2016-09-30,2016Q3,DB_Only,Other,10% to 20%,ACC,ACC_1995/1996,Legacy,No_Buyout,RAT/ROP,Q,R,False,0,0,6.0,72,EQH,EQH,2,313753.06,0.00,1261.574908,0.0,EI
4,2016,1,2016-01-29,2016Q1,DB_Only,Other,-10% to 0%,ACC,ACC_1995/1996,Legacy,No_Buyout,RAT/ROP,Q,R,False,0,0,6.0,<60,EQH,EQH,3,55486.66,0.00,103.699944,0.0,GAAP
5,2016,1,2016-01-29,2016Q1,DB_Only,Other,0% to 10%,ACC,ACC_1995/1996,Legacy,No_Buyout,RAT/ROP,NQ,R,False,0,0,6.0,84,EQH,EQH,2,450680.04,0.00,981.543570,0.0,GAAP
6,2016,2,2016-02-29,2016Q1,DB_Only,Other,30% to 40%,ACC,ACC_1995/1996,Legacy,No_Buyout,RAT/ROP,NQ,R,False,0,0,6.0,67,EQH,EQH,1,694885.40,0.00,1055.183480,0.0,GAAP
7,2016,1,2016-01-29,2016Q1,DB_Only,Other,0% to 10%,ACC,ACC_02,Legacy,No_Buyout,RAT/ROP,NQ,W,False,0,0,6.0,86+,EQH,EQH,18,2254907.37,746231.32,5080.738451,0.0,EI
8,2016,1,2016-01-29,2016Q1,GMIB,D4D,-50% and Less,ACC,ACC_02,Legacy,No_Buyout,RAT/ROP,NQ,W,False,0,0,6.0,83,EQH,EQH,1,22728.96,24476.75,16.694421,0.0,EI
9,2016,1,2016-01-29,2016Q1,GMIB,D4D,-10% to 0%,ACC,ACC_02,Legacy,No_Buyout,MAX/RU,Q,W,False,0,0,6.0,77,EQH,EQH,11,1725455.95,2348939.56,2171.630101,0.0,EI


In [11]:
# 10 highest total_precipitation samples
df.sort_values('', ascending=False).head(10)[['station_number', 'year', 'month', 'day', 'total_precipitation']]

KeyError: ignored

# Use BigQuery through pandas-gbq

The `pandas-gbq` library is a community led project by the pandas community. It covers basic functionality, such as writing a DataFrame to BigQuery and running a query, but as a third-party library it may not handle all BigQuery features or use cases.

[Pandas GBQ Documentation](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_gbq.html)

In [ ]:
import pandas as pd

sample_count = 2000
df = pd.io.gbq.read_gbq('''
  SELECT name, SUM(number) as count
  FROM `bigquery-public-data.usa_names.usa_1910_2013`
  WHERE state = 'TX'
  GROUP BY name
  ORDER BY count DESC
  LIMIT 100
''', project_id=project_id, dialect='standard')

df.head()

,name,count
0,James,272793
1,John,235139
2,Michael,225320
3,Robert,220399
4,David,219028


# Syntax highlighting
`google.colab.syntax` can be used to add syntax highlighting to any Python string literals which are used in a query later.

In [ ]:
from google.colab import syntax
query = syntax.sql('''
SELECT
  COUNT(*) as total_rows
FROM
  `bigquery-public-data.samples.gsod`
''')

pd.io.gbq.read_gbq(query, project_id=project_id, dialect='standard')

,total_rows
0,114420316
